# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [28]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from config import google_api_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_df = pd.read_csv('../WeatherPy/weather_check.csv')
weather_df.head()


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,harper,4.38,-7.72,77.72,88,99,5.28,LR,1603942061
1,1,barrow,71.29,-156.79,23.00,79,90,6.93,US,1603941974
2,2,east london,-33.02,27.91,68.27,84,96,6.31,ZA,1603941861
3,3,maniitsoq,65.42,-52.90,21.94,83,78,3.13,GL,1603942061
4,4,velikiy ustyug,60.76,46.31,42.01,93,100,6.44,RU,1603942061


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=google_api_key)

In [9]:
# Get the columns Lat and Lng for our location values
locations = weather_df[['Lat','Lng']].astype(float)
# Get humidty for our weight on the heatmap
humidity = weather_df['Humidity'].astype(float)

# Create the figure, create the heatmap layer
# Adjust the max intensity to be the humidity list
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, max_intensity = max(humidity), point_radius = 3, dissipating = False)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [24]:
# Set the parameters for ideal weather (we're rather picky)
idealtemp = [65,75]
idealhumidity = 70
idealwind = 7
idealclouds = 75
# Remove the temperatures that are too hot or too cold
ideal_weather_df = weather_df.loc[(weather_df['Max Temp']<idealtemp[1]) & (weather_df['Max Temp']>idealtemp[0]),:]

# Remove the places with too high of humidity (sorry Brazil)
ideal_weather_df = ideal_weather_df.loc[(ideal_weather_df['Humidity'] <= idealhumidity),:]

# Remove winds that are too high
ideal_weather_df = ideal_weather_df.loc[(ideal_weather_df['Wind Speed'] <= idealwind),:]

# Remove places that are too cloudy
ideal_weather_df = ideal_weather_df.loc[(ideal_weather_df['Cloudiness'] <= idealclouds),:]

#Check how many places we still have left after the culling
ideal_weather_df.shape

(17, 10)

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [44]:
hotel_df = ideal_weather_df
hotel_name = []

# Loop through the rows of the hotel dataframe and search for hotels within 5000m
for index,row in hotel_df.iterrows():
    
    params = {
        "location": "{0},{1}".format(row['Lat'],row['Lng']),
        "keyword": "hotel",
        "radius": 5000,
        "type": "lodging",
        "key": google_api_key
    }
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
    response = requests.get(base_url, params = params).json()
    
    # Try to put the hotel name into our list
    try:
        hotel_name.append(response['results'][0]['name'])
    # If we can't find a hotel let the user know which city is lacking
    except IndexError:
        print('Unfortunately, the city ' + str(row['City']) + 'has no hotels within 5000m')
        hotel_name.append('')
hotel_df['Hotel Name'] = hotel_name

Unfortunately, the city kalashas no hotels within 5000m
Unfortunately, the city mount darwinhas no hotels within 5000m
Unfortunately, the city ganzhouhas no hotels within 5000m
Unfortunately, the city miram shahhas no hotels within 5000m


In [43]:
hotel_name

['別邸 海と森',
 'Pullman Zamzam Makkah',
 'Lakefront Lodge Te Anau',
 'DB Star Hotel',
 'Waldorf Astoria Los Cabos Pedregal',
 'Hotel Alegria Gardens Amakusa',
 '',
 'Casa de la Aduana Hotel Boutique',
 '',
 'Luke Hotels',
 '',
 'HOSPEDAJE XIMENA',
 "Casa Turquesa Maison d'hôtes",
 'Nazuna 飫肥 城下町温泉 -小鹿倉邸-',
 'Kashuwa Lodge',
 '',
 'Sur Plaza Hotel']

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
